In [1]:
import QuantLib as ql

In [6]:
# setup dates
todaysDate = ql.Date(23,5,2023)
ql.Settings.instance().evaluationDate = todaysDate

# setup market data
euriborTermStructure = ql.RelinkableYieldTermStructureHandle()
euribor3m = ql.Euribor3M(euriborTermStructure)

cal = euribor3m.fixingCalendar()
fixingDays = euribor3m.fixingDays()
settlementDate = cal.advance(todaysDate, fixingDays, ql.Days, ql.ModifiedFollowing)
print(f'Today: {todaysDate.ISO()}')
print(f'settlement: {settlementDate.ISO()}')

Today: 2023-05-23
settlement: 2023-05-25


In [13]:
# setup 3 month term FRA quotes
fra1x4Rate =ql.QuoteHandle(ql.SimpleQuote(0.03))
fra2x5Rate =ql.QuoteHandle(ql.SimpleQuote(0.031))
fra3x6Rate =ql.QuoteHandle(ql.SimpleQuote(0.032))
fra6x9Rate =ql.QuoteHandle(ql.SimpleQuote(0.033))
fra9x12Rate =ql.QuoteHandle(ql.SimpleQuote(0.035))

In [20]:
# define Rate Helpers

fraDayCounter = euribor3m.dayCounter()
convention = euribor3m.businessDayConvention()
endOfMonth = euribor3m.endOfMonth()

fra1x4 = ql.FraRateHelper(fra1x4Rate,1,4,fixingDays, cal, convention, endOfMonth, fraDayCounter)
fra2x5 = ql.FraRateHelper(fra2x5Rate,2,5,fixingDays, cal, convention, endOfMonth, fraDayCounter)
fra3x6 = ql.FraRateHelper(fra3x6Rate,3,6,fixingDays, cal, convention, endOfMonth, fraDayCounter)
fra6x9 = ql.FraRateHelper(fra6x9Rate,6,9,fixingDays, cal, convention, endOfMonth, fraDayCounter)
fra9x12 = ql.FraRateHelper(fra9x12Rate,9,12,fixingDays, cal, convention, endOfMonth, fraDayCounter)

print(f'fra1x4 dates {fra1x4.earliestDate().ISO()} {fra1x4.latestDate().ISO()}')
print(f'fra2x5 dates {fra2x5.earliestDate().ISO()} {fra2x5.latestDate().ISO()}')
print(f'fra3x6 dates {fra3x6.earliestDate().ISO()} {fra3x6.latestDate().ISO()}')
print(f'fra6x9 dates {fra6x9.earliestDate().ISO()} {fra6x9.latestDate().ISO()}')
print(f'fra9x12 dates {fra9x12.earliestDate().ISO()} {fra9x12.latestDate().ISO()}')

fra1x4 dates 2023-06-26 2023-09-26
fra2x5 dates 2023-07-25 2023-10-25
fra3x6 dates 2023-08-25 2023-11-27
fra6x9 dates 2023-11-27 2024-02-27
fra9x12 dates 2024-02-26 2024-05-27


In [24]:
# Curve Buliding
tsDayCounter = ql.ActualActual(ql.ActualActual.ISDA)
fraInstruments = ql.RateHelperVector()
fraInstruments.append(fra1x4)
fraInstruments.append(fra2x5)
fraInstruments.append(fra3x6)
fraInstruments.append(fra6x9)
fraInstruments.append(fra9x12)

fraTermStructure = ql.PiecewiseLogLinearDiscount(todaysDate, fraInstruments, tsDayCounter)

discFactors=[]
for d in fraTermStructure.dates():
    date = d.ISO()
    timeFrom = fraTermStructure.timeFromReference(d)
    zero = fraTermStructure.zeroRate(d, ql.Actual365Fixed(), ql.Continuous)
    disc = fraTermStructure.discount(d)
    discData = {'Date': date,'TimeFrom':timeFrom,'ZeroRate':zero.rate()*100.0, 'Discount': disc}
    discFactors.append(discData)

In [25]:
import pandas as pd

discData = pd.DataFrame(discFactors)
display(discData)

/var/folders/n1/1xxjd2457hgd6l54vv_vdy_c0000gp/T/ipykernel_38069/3557948501.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


,Date,TimeFrom,ZeroRate,Discount
0,2023-05-23,0.000000,3.030066,1.000000
1,2023-09-26,0.345205,3.030066,0.989595
2,2023-10-25,0.424658,3.089780,0.986965
3,2023-11-27,0.515068,3.130516,0.984005
4,2024-02-27,0.766697,3.196653,0.975776
5,2024-05-27,1.012598,3.279022,0.967307


In [32]:
# Price the FRA

fraCalendar = euribor3m.fixingCalendar()
fraConvention = euribor3m.businessDayConvention()
fraFwdType = ql.Position.Long
fraNotional = 10000.0
fraTermMonths = 3
monthsToStart = [1,2,3,6,9]

euriborTermStructure.linkTo(fraTermStructure)

print('Test FRA construction, NPV calculation, and FRA purchase')

for m in monthsToStart:
    fraValueDate = fraCalendar.advance(settlementDate, m, ql.Months, fraConvention)
    fraStrikeRate = 0.03
    myFRA = ql.ForwardRateAgreement(euribor3m, fraValueDate, fraFwdType, fraStrikeRate, fraNotional)
    
    print(f'3M Term FRA, Months to start: {m}')
    print(f'strike FRA rate: {fraStrikeRate}')
    print(f'FRA 3M forward rate {myFRA.forwardRate().rate()}')
    print(f'FRA amount: {myFRA.amount()}')
    print(f'FRA NPV: {myFRA.NPV()}')

Test FRA construction, NPV calculation, and FRA purchase
3M Term FRA, Months to start: 1
strike FRA rate: 0.03
FRA 3M forward rate 0.02999999999999409
FRA amount: -1.4984537177315174e-11
FRA NPV: -1.4942302542542784e-11
3M Term FRA, Months to start: 2
strike FRA rate: 0.03
FRA 3M forward rate 0.03099999999997371
FRA amount: 2.535469006580696
FRA NPV: 2.5222431768404867
3M Term FRA, Months to start: 3
strike FRA rate: 0.03
FRA 3M forward rate 0.031999999999979184
FRA amount: 5.178949224205649
FRA NPV: 5.138692784655805
3M Term FRA, Months to start: 6
strike FRA rate: 0.03
FRA 3M forward rate 0.03299999999999984
FRA amount: 7.602551813042936
FRA NPV: 7.4809490068842575
3M Term FRA, Months to start: 9
strike FRA rate: 0.03
FRA 3M forward rate 0.035000000000000024
FRA amount: 12.528050442611956
FRA NPV: 12.225685191089083
